<a href="https://colab.research.google.com/github/rajagopal17/GenAI/blob/main/OpenAi_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#In this demo, the usage of OpenAI assistants functionality is provided.


*   data is stored in openai's vector database and queried from it (similiar to llamaindex)
*   we can create required number of assitants based on the required topic (each assistant queries its own vectorstore)


*   each assistant need to be created as a function with detailed function name using gpt-3.5/ gpt-4o
*   then create a tool list with  the assistants (function names)


*   using anthropic tool calling, required functions can be executed automatically
*   this is similiar to function calling discussed in another notebook, the only difference being, functions are created using openai assistants

#Points to check:


*   need to check how the token usage for retrieving data using gpt-3.5-turbo model from the vectorstore (cannot retrieve data using non-openai model)
*   also not sure where the vectorstore is stored in openai and will there be any privacy concerns


*   also we do not have any control on the vectorstore
*   if this is not cost effective we can use llamaindex with latest embeddings from openai/cohere/voyager/Ragatouille, create our own vectorstore and retrieve it using openai/claude models by creating a function wrapper on the retriever











In [ ]:
!pip install openai
import openai
import os
from openai import OpenAI as Client
from google.colab import userdata
os.environ["OPEN_API_KEY"] = userdata.get('OPEN_API_KEY')
openai.api_key = os.environ["OPEN_API_KEY"]
client     = Client(api_key= userdata.get('OPEN_API_KEY'))




---


#End-to-end process:

---




1.   Create a vector store and get the vector_store_id
2.   Create a assistant with vector_store.id, model and get assistant.id
3.   Create a function with three parameters


     *   thread
     *   message -> thread.id
     *   run -> assistant.id (assistant.id is linked to vector_stores.id)

First GB is free and beyond that, usage is billed at $0.10/GB/day of vector storage


---









#Create a vectore_store and save the vector_store.id
    -this is required to add in assistant creation

In [ ]:
sap_fixed_assets = client.beta.vector_stores.create(name="sap_fixed_assets")
file_path           = '/content/drive/MyDrive/Python/SAPDOCS/Copy of FixedAssets.pdf'
client.beta.vector_stores.files.upload(vector_store_id=sap_fixed_assets.id, file=open(file_path,'rb'))
sap_fixed_assets.id

'vs_BF5qMDKb6id1GJfaWUPO2LVD'

#Create assistant:


1.   Below two assistants are created with vectore_store.id
2.   print the assistant.id and add save it for later use



In [ ]:
#assistant-1
fixed_assets_assistant = client.beta.assistants.create(
                                          name          = "fixed_assets_accountant",
                                          description   = '''
                                                          you are a SAP consultant with expertize in configuration of fixed assets
                                                          and financial accounting process. Analyze the user query and provide
                                                          the required details from the knowledge base ''' ,
                                          model         = "gpt-3.5-turbo-0125",
                                          tools         = [{"type": "file_search"}],
                                          tool_resources= {"file_search": {"vector_store_ids":['vs_BF5qMDKb6id1GJfaWUPO2LVD']}}
                                                            )



#assistant-2
depreciation_assistant = client.beta.assistants.create(
                                          name          = "depreciation_accountant",
                                          description   = '''
                                                          you are a SAP consultant with expertize in configuration of fixed assets
                                                          and depreciation process. Analyze the user query and provide
                                                          the required details from the knowledge base ''' ,
                                          model         = "gpt-3.5-turbo-0125",
                                          tools         = [{"type": "file_search"}],
                                          tool_resources= {"file_search": {"vector_store_ids":['vs_BF5qMDKb6id1GJfaWUPO2LVD']}}
                                                            )




#Run assistants by adding respective assistant.id

In [ ]:
import textwrap

def get_fixed_asset_details(user_query):
        thread   = client.beta.threads.create()
        message  = client.beta.threads.messages.create(
                          thread_id    = thread.id,
                          role         = "user",
                          content      = user_query)
        run      = client.beta.threads.runs.create_and_poll(
                          thread_id    =thread.id,
                          assistant_id = 'asst_galXOLYJ1L3Fx2BM3SH6fW5y',
                          instructions ='''Please address the user as Sirji. Provide output in a reader friendly format''' )


        messages = client.beta.threads.messages.list(thread_id=thread.id)
        if run.status == 'completed':
            messages   = client.beta.threads.messages.list(thread_id=thread.id)
            return (messages.data[0].content[0].text.value)
        else:
            return (run.status)



In [ ]:
print(get_fixed_asset_details('can you explain in detail what is use of plant field in asset master'))

In the asset master record, the "plant" field serves as an organizational unit primarily defined in SAP R/3 logistics components. Here is the detailed information regarding the use of the plant field in the asset master:

- **Use**:
  - The plant field has no direct asset accounting relevance but can be utilized as a sorting and selection criterion for reports.
  - It allows the assignment of a fixed asset to a specific plant for a defined period within its asset master record.
  - Changes in the assignment to a different plant can be made by modifying the asset master record.

- **Prerequisites**:
  - It is necessary to maintain plants and locations in Customizing by selecting Enterprise Structure.

- **Features**:
  - **Plant**:
    - Typically represents a plant location or branch.
    - Although not directly impacting asset accounting, it can be used for sorting and selecting reports.
  - **Location**:
    - Handled similarly to the plant, allowing for time-dependent assignment of 

In [ ]:
print(get_fixed_asset_details('can you explain what is use of account determination field in asset master')) #gpt-4o

The account determination field in the asset master is used to define the reconciliation accounts in Financial Accounting that should be posted during asset transactions. This field is entered in the general master data for the asset class and ensures consistent account assignment for all assets within the given asset class. The account determination key is effective for each chart of accounts and for each depreciation area defined as an automatic posting area in the respective chart of depreciation【4:0†source】.


In [ ]:
#Required_parameters are : depreciation_assistant.id =asst_eahH1Kow1azpKVZ91VTCjeO9


def get_depreciation_details(user_query):

      thread   = client.beta.threads.create()
      message  = client.beta.threads.messages.create(
                          thread_id    = thread.id,
                          role         = "user",
                          content      = user_query)
      run      = client.beta.threads.runs.create_and_poll(
                          thread_id    =thread.id,
                          assistant_id = 'asst_eahH1Kow1azpKVZ91VTCjeO9',
                          instructions ='''Please address the user as Sirji. Provide output in a reader friendly format''' )


      messages = client.beta.threads.messages.list(thread_id=thread.id)
      if  run.status == 'completed':
          messages   = client.beta.threads.messages.list(thread_id=thread.id)
          return (messages.data[0].content[0].text.value)
      else:
          return (run.status)

#print(get_depreciation_details('can you explain the different methods of depreciation used in SAP'))

In [ ]:
user_query ="what is the purpose of time dependent field in asset master"
print(textwrap.fill(get_depreciation_details(user_query),width=150,break_long_words=False,replace_whitespace=False))

Sirji, the purpose of the time-dependent field in asset master is to track changes in certain assignments during the life cycle of an asset that
change frequently. This field allows for the recording of changes for reporting and valuation needs, such as changes in assignments like cost center,
plant, activity type, and more with different time intervals within the asset master record. Data is stored in a sequence with valid from and valid to
dates, showing the exact day of the change, and allowing the system to display the time-dependent data valid for the date of the request【4:0†source】.
